In [1]:
from pprint import pprint
from IPython.display import display

from fda_ode import *

In [2]:
t, h, k = symbols(r't, h, k', real=True)
p, q, r = (f(t) for f in symbols('p, q, r', cls=Function))

## dx xn+1 − xn F (xn+1 ) + F (xn ) = F (x) ⇒ = dt ∆t 2 

Theorem (Cooper, 1987) The implicit midpoint rule automatically inherits each quadratic conservation law. If the field of algebraical integrals of dynamic system is generated by quadratic forms, then the implicit midpoint rule is total conservative. Ref.: Sanz-Serna J.M. // SIAM Review. 2016. Vol. 58, No. 1, pp. 3–33.

In [3]:
s = {
    p.diff(t): q*r,
    q.diff(t): -p*r,
    r.diff(t): -k**2*p*q,
}

In [4]:
(p**2 + q**2).diff(t).subs(s).simplify()

0

In [5]:
(k**2*p**2 + r**2).diff(t).subs(s).simplify()

0

In [6]:
init((p, q, r), t, h)

In [7]:
def GaussLegendre2(f, y):
    y1 = Matrix([T(m, 1) for m in y])
    k1 = f(y1/2 + y/2)
    return expand(k1)

In [8]:
def CrankNicolson2(f, y):
    y1 = Matrix([T(m, 1) for m in y])
    k1 = f(y)
    k2 = f(y1)
    return expand((k1 + k2)/2)

In [9]:
def RungeKutta4(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/2)
    k3 = f(y + h*k2/2)
    k4 = f(y + h*k3)
    return expand((k1 + 2*k2 + 2*k3 + k4)/6)

In [10]:
def DormandPrince5(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/5)
    k3 = f(y + h*(k1*3/40 + k2*9/40))
    k4 = f(y + h*(k1*44/45 - k2*56/15 + k3*32/9))
    k5 = f(y + h*(k1*19372/6561 - k2*25360/2187 + k3*64448/6561 - k4*212/729))
    k6 = f(y + h*(k1*9017/3168 - k2*355/33 + k3*46732/5247 + k4*49/176 - k5*5103/18656))
    return expand(k1*35/384 + k3*500/1113 + k4*125/192 - k5*2187/6784 + k6*11/84)

def DormandPrince51(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/5)
    k3 = f(y + h*(k1*3/40 + k2*9/40))
    k4 = f(y + h*(k1*44/45 - k2*56/15 + k3*32/9))
    k5 = f(y + h*(k1*19372/6561 - k2*25360/2187 + k3*64448/6561 - k4*212/729))
    k6 = f(y + h*(k1*9017/3168 - k2*355/33 + k3*46732/5247 + k4*49/176 - k5*5103/18656))
    k7 = f(y + h*(k1*35/384 + k3*500/1113 + k4*125/192 - k5*2187/6784 + k6*11/84))
    return expand(k1*5179/57600 + k3*7571/16695 + k4*393/640 - k5*92097/339200 + k6*187/2100 + k7/40)

In [11]:
def f(y):
    return Matrix([\
        clip_n(y[1]*y[2]),\
        clip_n(-y[0]*y[2]),\
        clip_n(-k**2*y[0]*y[1]),\
    ])
    
s = (('p(t)', 'y[0]'), ('q(t)', 'y[1]'), ('r(t)', 'y[2]'))
def replace(r):
    for a, b in s:
        r = r.replace(a, b)
    return r

## Gauss–Legendre method

In [12]:
set_clip(8, 7, Rational(0, 1))
m = GaussLegendre2(f, Matrix([p, q, r]))

In [13]:
F1 = clip((T(p, 1)-T(p, 0))/h - T(m[0], 0))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-(-p_{tt} + q*r_{t} + q_{t}*r)/2

h^2 =>


...

h^5 =>


-(-2*p_{tttttt} + 6*q*r_{ttttt} + 6*q_{ttttt}*r + 15*q_{tttt}*r_{t} + 30*q_{ttt}*r_{tt} + 30*q_{tt}*r_{ttt} + 15*q_{t}*r_{tttt})/1440

h^6 =>


-(-4*p_{ttttttt} + 14*q*r_{tttttt} + 14*q_{tttttt}*r + 42*q_{ttttt}*r_{t} + 105*q_{tttt}*r_{tt} + 140*q_{ttt}*r_{ttt} + 105*q_{tt}*r_{tttt} + 42*q_{t}*r_{ttttt})/20160

p_{t} - q r
+h\big(
- \frac{- p_{tt} + q r_{t} + q_{t} r}{2}
\big)
+h^2\big(
- \frac{- 2 p_{ttt} + 3 q r_{tt} + 3 q_{tt} r + 3 q_{t} r_{t}}{12}
\big)
+h^3\big(
- \frac{- p_{tttt} + 2 q r_{ttt} + 2 q_{ttt} r + 3 q_{tt} r_{t} + 3 q_{t} r_{tt}}{24}
\big)
+h^4\big(
- \frac{- 2 p_{ttttt} + 5 q r_{tttt} + 5 q_{tttt} r + 10 q_{ttt} r_{t} + 15 q_{tt} r_{tt} + 10 q_{t} r_{ttt}}{240}
\big)
+h^5\big(
- \frac{- 2 p_{tttttt} + 6 q r_{ttttt} + 6 q_{ttttt} r + 15 q_{tttt} r_{t} + 30 q_{ttt} r_{tt} + 30 q_{tt} r_{ttt} + 15 q_{t} r_{tttt}}{1440}
\big)
+h^6\big(
- \frac{- 4 p_{ttttttt} + 14 q r_{tttttt} + 14 q_{tttttt} r + 42 q_{ttttt} r_{t} + 105 q_{tttt} r_{tt} + 140 q_{ttt} r_{ttt} + 105 q_{tt} r_{tttt} + 42 q_{t} r_{ttttt}}{20160}
\big)


In [14]:
F2 = clip((T(q, 1)-T(q, 0))/h - T(m[1], 0))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


(p*r_{t} + p_{t}*r + q_{tt})/2

h^2 =>


...

h^5 =>


(6*p*r_{ttttt} + 6*p_{ttttt}*r + 15*p_{tttt}*r_{t} + 30*p_{ttt}*r_{tt} + 30*p_{tt}*r_{ttt} + 15*p_{t}*r_{tttt} + 2*q_{tttttt})/1440

h^6 =>


(14*p*r_{tttttt} + 14*p_{tttttt}*r + 42*p_{ttttt}*r_{t} + 105*p_{tttt}*r_{tt} + 140*p_{ttt}*r_{ttt} + 105*p_{tt}*r_{tttt} + 42*p_{t}*r_{ttttt} + 4*q_{ttttttt})/20160

p r + q_{t}
+h\big(
\frac{p r_{t} + p_{t} r + q_{tt}}{2}
\big)
+h^2\big(
\frac{3 p r_{tt} + 3 p_{tt} r + 3 p_{t} r_{t} + 2 q_{ttt}}{12}
\big)
+h^3\big(
\frac{2 p r_{ttt} + 2 p_{ttt} r + 3 p_{tt} r_{t} + 3 p_{t} r_{tt} + q_{tttt}}{24}
\big)
+h^4\big(
\frac{5 p r_{tttt} + 5 p_{tttt} r + 10 p_{ttt} r_{t} + 15 p_{tt} r_{tt} + 10 p_{t} r_{ttt} + 2 q_{ttttt}}{240}
\big)
+h^5\big(
\frac{6 p r_{ttttt} + 6 p_{ttttt} r + 15 p_{tttt} r_{t} + 30 p_{ttt} r_{tt} + 30 p_{tt} r_{ttt} + 15 p_{t} r_{tttt} + 2 q_{tttttt}}{1440}
\big)
+h^6\big(
\frac{14 p r_{tttttt} + 14 p_{tttttt} r + 42 p_{ttttt} r_{t} + 105 p_{tttt} r_{tt} + 140 p_{ttt} r_{ttt} + 105 p_{tt} r_{tttt} + 42 p_{t} r_{ttttt} + 4 q_{ttttttt}}{20160}
\big)


In [15]:
F3 = clip((T(r, 1)-T(r, 0))/h - T(m[2], 0))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


k**2*(p*q_{t} + p_{t}*q)/2 + r_{tt}/2

h^2 =>


...

h^5 =>


k**2*(2*p*q_{ttttt} + 2*p_{ttttt}*q + 5*p_{tttt}*q_{t} + 10*p_{ttt}*q_{tt} + 10*p_{tt}*q_{ttt} + 5*p_{t}*q_{tttt})/480 + r_{tttttt}/720

h^6 =>


k**2*(2*p*q_{tttttt} + 2*p_{tttttt}*q + 6*p_{ttttt}*q_{t} + 15*p_{tttt}*q_{tt} + 20*p_{ttt}*q_{ttt} + 15*p_{tt}*q_{tttt} + 6*p_{t}*q_{ttttt})/2880 + r_{ttttttt}/5040

k^{2} p q + r_{t}
+h\big(
\frac{k^{2} \left(p q_{t} + p_{t} q\right)}{2} + \frac{r_{tt}}{2}
\big)
+h^2\big(
\frac{k^{2} \left(p q_{tt} + p_{tt} q + p_{t} q_{t}\right)}{4} + \frac{r_{ttt}}{6}
\big)
+h^3\big(
\frac{k^{2} \cdot \left(2 p q_{ttt} + 2 p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{24} + \frac{r_{tttt}}{24}
\big)
+h^4\big(
\frac{k^{2} \left(p q_{tttt} + p_{tttt} q + 2 p_{ttt} q_{t} + 3 p_{tt} q_{tt} + 2 p_{t} q_{ttt}\right)}{48} + \frac{r_{ttttt}}{120}
\big)
+h^5\big(
\frac{k^{2} \cdot \left(2 p q_{ttttt} + 2 p_{ttttt} q + 5 p_{tttt} q_{t} + 10 p_{ttt} q_{tt} + 10 p_{tt} q_{ttt} + 5 p_{t} q_{tttt}\right)}{480} + \frac{r_{tttttt}}{720}
\big)
+h^6\big(
\frac{k^{2} \cdot \left(2 p q_{tttttt} + 2 p_{tttttt} q + 6 p_{ttttt} q_{t} + 15 p_{tttt} q_{tt} + 20 p_{ttt} q_{ttt} + 15 p_{tt} q_{tttt} + 6 p_{t} q_{ttttt}\right)}{2880} + \frac{r_{ttttttt}}{5040}
\big)


In [16]:
f1 = NF(F1, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


-k**6*q*r*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/6720 + k**4*q*r**3*(29*p**4 + 4*p**2*q**2 + 29*q**4)/3360 - 29*k**2*q*r**5*(p - q)*(p + q)/3360 + q*r**7/448

p_{t} - q r
+h\big(
0
\big)
+h^2\big(
- \frac{k^{2} q r \left(p - q\right) \left(p + q\right)}{12} + \frac{q r^{3}}{12}
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{k^{4} q r \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{240} + \frac{7 k^{2} q r^{3} \left(p - q\right) \left(p + q\right)}{240} - \frac{q r^{5}}{80}
\big)
+h^5\big(
0
\big)
+h^6\big(
- \frac{k^{6} q r \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{6720} + \frac{k^{4} q r^{3} \cdot \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{3360} - \frac{29 k^{2} q r^{5} \left(p - q\right) \left(p + q\right)}{3360} + \frac{q r^{7}}{448}
\big)


In [17]:
f2 = NF(F2, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


k**6*p*r*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/6720 - k**4*p*r**3*(29*p**4 + 4*p**2*q**2 + 29*q**4)/3360 + 29*k**2*p*r**5*(p - q)*(p + q)/3360 - p*r**7/448

p r + q_{t}
+h\big(
0
\big)
+h^2\big(
\frac{k^{2} p r \left(p - q\right) \left(p + q\right)}{12} - \frac{p r^{3}}{12}
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{4} p r \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{240} - \frac{7 k^{2} p r^{3} \left(p - q\right) \left(p + q\right)}{240} + \frac{p r^{5}}{80}
\big)
+h^5\big(
0
\big)
+h^6\big(
\frac{k^{6} p r \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{6720} - \frac{k^{4} p r^{3} \cdot \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{3360} + \frac{29 k^{2} p r^{5} \left(p - q\right) \left(p + q\right)}{3360} - \frac{p r^{7}}{448}
\big)


In [18]:
f3 = NF(F3, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


k**8*p*q*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/6720 - k**6*p*q*r**2*(29*p**4 + 4*p**2*q**2 + 29*q**4)/3360 + 29*k**4*p*q*r**4*(p - q)*(p + q)/3360 - k**2*p*q*r**6/448

k^{2} p q + r_{t}
+h\big(
0
\big)
+h^2\big(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{12} - \frac{k^{2} p q r^{2}}{12}
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{6} p q \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{240} - \frac{7 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{240} + \frac{k^{2} p q r^{4}}{80}
\big)
+h^5\big(
0
\big)
+h^6\big(
\frac{k^{8} p q \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{6720} - \frac{k^{6} p q r^{2} \cdot \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{3360} + \frac{29 k^{4} p q r^{4} \left(p - q\right) \left(p + q\right)}{3360} - \frac{k^{2} p q r^{6}}{448}
\big)


In [19]:
F4 = clip(((T(p, 1)**2 + T(q, 1)**2) -\
           (T(p, 0)**2 + T(q, 0)**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

h^6 =>


(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt} + q*q_{ttttttt} + 7*q_{tttttt}*q_{t} + 21*q_{ttttt}*q_{tt} + 35*q_{tttt}*q_{ttt})/2520

2 \left(p p_{t} + q q_{t}\right)
+h\big(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\big)
+h^2\big(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\big)
+h^3\big(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\big)
+h^5\big(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\big)
+h^6\big(
\frac{p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt} + q q_{ttttttt} + 7 q_{tttttt} q_{t} + 21 q_{ttttt} q_{tt} + 35 q_{tttt} q_{ttt}}{2520}
\big)


In [20]:
f4 = NF(F4, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
0
\big)
+h^6\big(
0
\big)


In [21]:
F5 = clip(((k**2*T(p, 1)**2 + T(r, 1)**2) -\
           (k**2*T(p, 0)**2 + T(r, 0)**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

h^6 =>


k**2*(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt})/2520 + (r*r_{ttttttt} + 7*r_{tttttt}*r_{t} + 21*r_{ttttt}*r_{tt} + 35*r_{tttt}*r_{ttt})/2520

2 k^{2} p p_{t} + 2 r r_{t}
+h\big(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\big)
+h^2\big(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\big)
+h^3\big(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\big)
+h^5\big(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\big)
+h^6\big(
\frac{k^{2} \left(p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt}\right)}{2520} + \frac{r r_{ttttttt} + 7 r_{tttttt} r_{t} + 21 r_{ttttt} r_{tt} + 35 r_{tttt} r_{ttt}}{2520}
\big)


In [22]:
f5 = NF(F5, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
0
\big)
+h^6\big(
0
\big)


## Crank-Nicolson method

In [23]:
set_clip(5, 4, Rational(0, 1))
m = CrankNicolson2(f, Matrix([p, q, r]))

In [24]:
F1 = clip((T(p, 1)-T(p, 0))/h - T(m[0], 0))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-(-p_{tt} + q*r_{t} + q_{t}*r)/2

h^2 =>


-(-2*p_{ttt} + 3*q*r_{tt} + 3*q_{tt}*r + 6*q_{t}*r_{t})/12

h^3 =>


-(-p_{tttt} + 2*q*r_{ttt} + 2*q_{ttt}*r + 6*q_{tt}*r_{t} + 6*q_{t}*r_{tt})/24

p_{t} - q r
+h\big(
- \frac{- p_{tt} + q r_{t} + q_{t} r}{2}
\big)
+h^2\big(
- \frac{- 2 p_{ttt} + 3 q r_{tt} + 3 q_{tt} r + 6 q_{t} r_{t}}{12}
\big)
+h^3\big(
- \frac{- p_{tttt} + 2 q r_{ttt} + 2 q_{ttt} r + 6 q_{tt} r_{t} + 6 q_{t} r_{tt}}{24}
\big)


In [25]:
F2 = clip((T(q, 1)-T(q, 0))/h - T(m[1], 0))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


(p*r_{t} + p_{t}*r + q_{tt})/2

h^2 =>


(3*p*r_{tt} + 3*p_{tt}*r + 6*p_{t}*r_{t} + 2*q_{ttt})/12

h^3 =>


(2*p*r_{ttt} + 2*p_{ttt}*r + 6*p_{tt}*r_{t} + 6*p_{t}*r_{tt} + q_{tttt})/24

p r + q_{t}
+h\big(
\frac{p r_{t} + p_{t} r + q_{tt}}{2}
\big)
+h^2\big(
\frac{3 p r_{tt} + 3 p_{tt} r + 6 p_{t} r_{t} + 2 q_{ttt}}{12}
\big)
+h^3\big(
\frac{2 p r_{ttt} + 2 p_{ttt} r + 6 p_{tt} r_{t} + 6 p_{t} r_{tt} + q_{tttt}}{24}
\big)


In [26]:
F3 = clip((T(r, 1)-T(r, 0))/h - T(m[2], 0))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


k**2*(p*q_{t} + p_{t}*q)/2 + r_{tt}/2

h^2 =>


k**2*(p*q_{tt} + p_{tt}*q + 2*p_{t}*q_{t})/4 + r_{ttt}/6

h^3 =>


k**2*(p*q_{ttt} + p_{ttt}*q + 3*p_{tt}*q_{t} + 3*p_{t}*q_{tt})/12 + r_{tttt}/24

k^{2} p q + r_{t}
+h\big(
\frac{k^{2} \left(p q_{t} + p_{t} q\right)}{2} + \frac{r_{tt}}{2}
\big)
+h^2\big(
\frac{k^{2} \left(p q_{tt} + p_{tt} q + 2 p_{t} q_{t}\right)}{4} + \frac{r_{ttt}}{6}
\big)
+h^3\big(
\frac{k^{2} \left(p q_{ttt} + p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{12} + \frac{r_{tttt}}{24}
\big)


In [27]:
f1 = NF(F1, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


-k**2*q*r*(2*p - q)*(2*p + q)/12 + q*r**3/12

h^3 =>


0

p_{t} - q r
+h\big(
0
\big)
+h^2\big(
- \frac{k^{2} q r \left(2 p - q\right) \left(2 p + q\right)}{12} + \frac{q r^{3}}{12}
\big)
+h^3\big(
0
\big)


In [28]:
f2 = NF(F2, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


k**2*p*r*(p - 2*q)*(p + 2*q)/12 - p*r**3/12

h^3 =>


0

p r + q_{t}
+h\big(
0
\big)
+h^2\big(
\frac{k^{2} p r \left(p - 2 q\right) \left(p + 2 q\right)}{12} - \frac{p r^{3}}{12}
\big)
+h^3\big(
0
\big)


In [29]:
f3 = NF(F3, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


k**4*p*q*(p - q)*(p + q)/12 - k**2*p*q*r**2/3

h^3 =>


0

k^{2} p q + r_{t}
+h\big(
0
\big)
+h^2\big(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{12} - \frac{k^{2} p q r^{2}}{3}
\big)
+h^3\big(
0
\big)


In [30]:
F4 = clip(((T(p, 1)**2 + T(q, 1)**2) -\
           (T(p, 0)**2 + T(q, 0)**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


(p*p_{ttt} + 3*p_{tt}*p_{t} + q*q_{ttt} + 3*q_{tt}*q_{t})/3

h^3 =>


(p*p_{tttt} + 4*p_{ttt}*p_{t} + 3*p_{tt}**2 + q*q_{tttt} + 4*q_{ttt}*q_{t} + 3*q_{tt}**2)/12

2 \left(p p_{t} + q q_{t}\right)
+h\big(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\big)
+h^2\big(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\big)
+h^3\big(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\big)


In [31]:
f4 = NF(F4, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


k**2*p*q*r*(p**2 + q**2)/2

h^3 =>


-k**4*p**2*q**2*(p**2 + q**2)/4 - k**2*r**2*(p - q)*(p + q)*(p**2 + q**2)/4

0
+h\big(
0
\big)
+h^2\big(
\frac{k^{2} p q r \left(p^{2} + q^{2}\right)}{2}
\big)
+h^3\big(
- \frac{k^{4} p^{2} q^{2} \left(p^{2} + q^{2}\right)}{4} - \frac{k^{2} r^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{4}
\big)


In [32]:
replace(str(f4[2].collect(k)))

'k**2*(y[0]**3*y[1]*y[2]/2 + y[0]*y[1]**3*y[2]/2)'

In [33]:
F5 = clip(((k**2*T(p, 1)**2 + T(r, 1)**2) -\
           (k**2*T(p, 0)**2 + T(r, 0)**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


k**2*(p*p_{ttt} + 3*p_{tt}*p_{t})/3 + (r*r_{ttt} + 3*r_{tt}*r_{t})/3

h^3 =>


k**2*(p*p_{tttt} + 4*p_{ttt}*p_{t} + 3*p_{tt}**2)/12 + (r*r_{tttt} + 4*r_{ttt}*r_{t} + 3*r_{tt}**2)/12

2 k^{2} p p_{t} + 2 r r_{t}
+h\big(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\big)
+h^2\big(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\big)
+h^3\big(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\big)


In [34]:
f5 = NF(F5, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


k**4*p**3*q*r/2 + k**2*p*q*r**3/2

h^3 =>


-k**6*p**4*q**2/4 - k**4*p**4*r**2/4 - k**2*r**4*(p - q)*(p + q)/4

0
+h\big(
0
\big)
+h^2\big(
\frac{k^{4} p^{3} q r}{2} + \frac{k^{2} p q r^{3}}{2}
\big)
+h^3\big(
- \frac{k^{6} p^{4} q^{2}}{4} - \frac{k^{4} p^{4} r^{2}}{4} - \frac{k^{2} r^{4} \left(p - q\right) \left(p + q\right)}{4}
\big)


In [35]:
replace(str(f5[2].factor()))

'k**2*(k**2*y[0]**2 + y[2]**2)*y[0]*y[1]*y[2]/2'

## "original" Runge–Kutta method

In [36]:
set_clip(7, 6, Rational(0, 1))
m = RungeKutta4(f, Matrix([p, q, r]))

In [37]:
F1 = clip((T(p, 1)-T(p, 0))/h - T(m[0], 0))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


k**2*p*q**2/2 + (p*r**2 + p_{tt})/2

h^2 =>


...

h^4 =>


-k**4*p**2*q*r*(7*p**2 - 17*q**2)/48 + k**2*q*r**3*(17*p**2 - 5*q**2)/48 + p_{ttttt}/120

h^5 =>


k**6*p**3*q**2*(2*p**2 - 5*q**2)/96 + k**4*p*r**2*(p**4 - 19*p**2*q**2 + 7*q**4)/48 - k**2*p*r**4*(5*p**2 - 14*q**2)/96 + p_{tttttt}/720

p_{t} - q r
+h\big(
\frac{k^{2} p q^{2}}{2} + \frac{p r^{2} + p_{tt}}{2}
\big)
+h^2\big(
- \frac{k^{2} q r \left(2 p - q\right) \left(2 p + q\right)}{6} + \frac{p_{ttt} + q r^{3}}{6}
\big)
+h^3\big(
\frac{k^{4} p q^{2} \cdot \left(2 p - q\right) \left(2 p + q\right)}{24} + \frac{k^{2} p r^{2} \cdot \left(2 p^{2} - 7 q^{2}\right)}{12} - \frac{p r^{4} - p_{tttt}}{24}
\big)
+h^4\big(
- \frac{k^{4} p^{2} q r \left(7 p^{2} - 17 q^{2}\right)}{48} + \frac{k^{2} q r^{3} \cdot \left(17 p^{2} - 5 q^{2}\right)}{48} + \frac{p_{ttttt}}{120}
\big)
+h^5\big(
\frac{k^{6} p^{3} q^{2} \cdot \left(2 p^{2} - 5 q^{2}\right)}{96} + \frac{k^{4} p r^{2} \left(p^{4} - 19 p^{2} q^{2} + 7 q^{4}\right)}{48} - \frac{k^{2} p r^{4} \cdot \left(5 p^{2} - 14 q^{2}\right)}{96} + \frac{p_{tttttt}}{720}
\big)


In [38]:
F2 = clip((T(q, 1)-T(q, 0))/h - T(m[1], 0))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


-k**2*p**2*q/2 + (q*r**2 + q_{tt})/2

h^2 =>


...

h^4 =>


-k**4*p*q**2*r*(17*p**2 - 7*q**2)/48 - k**2*p*r**3*(5*p**2 - 17*q**2)/48 + q_{ttttt}/120

h^5 =>


k**6*p**2*q**3*(5*p**2 - 2*q**2)/96 + k**4*q*r**2*(7*p**4 - 19*p**2*q**2 + q**4)/48 - k**2*q*r**4*(14*p**2 - 5*q**2)/96 + q_{tttttt}/720

p r + q_{t}
+h\big(
- \frac{k^{2} p^{2} q}{2} + \frac{q r^{2} + q_{tt}}{2}
\big)
+h^2\big(
\frac{k^{2} p r \left(p - 2 q\right) \left(p + 2 q\right)}{6} - \frac{p r^{3} - q_{ttt}}{6}
\big)
+h^3\big(
- \frac{k^{4} p^{2} q \left(p - 2 q\right) \left(p + 2 q\right)}{24} + \frac{k^{2} q r^{2} \cdot \left(7 p^{2} - 2 q^{2}\right)}{12} - \frac{q r^{4} - q_{tttt}}{24}
\big)
+h^4\big(
- \frac{k^{4} p q^{2} r \left(17 p^{2} - 7 q^{2}\right)}{48} - \frac{k^{2} p r^{3} \cdot \left(5 p^{2} - 17 q^{2}\right)}{48} + \frac{q_{ttttt}}{120}
\big)
+h^5\big(
\frac{k^{6} p^{2} q^{3} \cdot \left(5 p^{2} - 2 q^{2}\right)}{96} + \frac{k^{4} q r^{2} \cdot \left(7 p^{4} - 19 p^{2} q^{2} + q^{4}\right)}{48} - \frac{k^{2} q r^{4} \cdot \left(14 p^{2} - 5 q^{2}\right)}{96} + \frac{q_{tttttt}}{720}
\big)


In [39]:
F3 = clip((T(r, 1)-T(r, 0))/h - T(m[2], 0))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


-k**2*r*(p - q)*(p + q)/2 + r_{tt}/2

h^2 =>


...

h^4 =>


-5*k**6*p**3*q**3/48 - 17*k**4*p*q*r**2*(p - q)*(p + q)/48 + 7*k**2*p*q*r**4/48 + r_{ttttt}/120

h^5 =>


7*k**6*p**2*q**2*r*(p - q)*(p + q)/48 + k**4*r**3*(5*p**4 - 38*p**2*q**2 + 5*q**4)/96 - k**2*r**5*(p - q)*(p + q)/48 + r_{tttttt}/720

k^{2} p q + r_{t}
+h\big(
- \frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2} + \frac{r_{tt}}{2}
\big)
+h^2\big(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{6} - \frac{2 k^{2} p q r^{2}}{3} + \frac{r_{ttt}}{6}
\big)
+h^3\big(
- \frac{k^{4} r \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{24} + \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{6} + \frac{r_{tttt}}{24}
\big)
+h^4\big(
- \frac{5 k^{6} p^{3} q^{3}}{48} - \frac{17 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{48} + \frac{7 k^{2} p q r^{4}}{48} + \frac{r_{ttttt}}{120}
\big)
+h^5\big(
\frac{7 k^{6} p^{2} q^{2} r \left(p - q\right) \left(p + q\right)}{48} + \frac{k^{4} r^{3} \cdot \left(5 p^{4} - 38 p^{2} q^{2} + 5 q^{4}\right)}{96} - \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{48} + \frac{r_{tttttt}}{720}
\big)


In [40]:
f1 = NF(F1, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


-k**4*q*r*(3*p**4 + 3*p**2*q**2 - 2*q**4)/240 - k**2*q*r**3*(p - q)*(p + q)/80 + q*r**5/120

h^5 =>


-k**6*p*q**2*(p - q)*(p + q)*(p**2 + 2*q**2)/288 - k**4*p*r**2*(p**4 - 10*p**2*q**2 - 3*q**4)/288 - k**2*p*r**4*(p**2 - 3*q**2)/288 + p*r**6/144

p_{t} - q r
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{k^{4} q r \left(3 p^{4} + 3 p^{2} q^{2} - 2 q^{4}\right)}{240} - \frac{k^{2} q r^{3} \left(p - q\right) \left(p + q\right)}{80} + \frac{q r^{5}}{120}
\big)
+h^5\big(
- \frac{k^{6} p q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + 2 q^{2}\right)}{288} - \frac{k^{4} p r^{2} \left(p^{4} - 10 p^{2} q^{2} - 3 q^{4}\right)}{288} - \frac{k^{2} p r^{4} \left(p^{2} - 3 q^{2}\right)}{288} + \frac{p r^{6}}{144}
\big)


In [41]:
f2 = NF(F2, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-k**4*p*r*(2*p**4 - 3*p**2*q**2 - 3*q**4)/240 + k**2*p*r**3*(p - q)*(p + q)/80 - p*r**5/120

h^5 =>


-k**6*p**2*q*(p - q)*(p + q)*(2*p**2 + q**2)/288 + k**4*q*r**2*(3*p**4 + 10*p**2*q**2 - q**4)/288 - k**2*q*r**4*(3*p**2 - q**2)/288 + q*r**6/144

p r + q_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{k^{4} p r \left(2 p^{4} - 3 p^{2} q^{2} - 3 q^{4}\right)}{240} + \frac{k^{2} p r^{3} \left(p - q\right) \left(p + q\right)}{80} - \frac{p r^{5}}{120}
\big)
+h^5\big(
- \frac{k^{6} p^{2} q \left(p - q\right) \left(p + q\right) \left(2 p^{2} + q^{2}\right)}{288} + \frac{k^{4} q r^{2} \cdot \left(3 p^{4} + 10 p^{2} q^{2} - q^{4}\right)}{288} - \frac{k^{2} q r^{4} \cdot \left(3 p^{2} - q^{2}\right)}{288} + \frac{q r^{6}}{144}
\big)


In [42]:
f3 = NF(F3, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-k**6*p*q*(2*p**4 - 3*p**2*q**2 + 2*q**4)/240 + k**4*p*q*r**2*(p - q)*(p + q)/80 + k**2*p*q*r**4/80

h^5 =>


-k**6*r*(p - q)*(p + q)*(2*p**4 - p**2*q**2 + 2*q**4)/288 + k**4*r**3*(p**4 + 10*p**2*q**2 + q**4)/288 + k**2*r**5*(p - q)*(p + q)/288

k^{2} p q + r_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{k^{6} p q \left(2 p^{4} - 3 p^{2} q^{2} + 2 q^{4}\right)}{240} + \frac{k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{80} + \frac{k^{2} p q r^{4}}{80}
\big)
+h^5\big(
- \frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(2 p^{4} - p^{2} q^{2} + 2 q^{4}\right)}{288} + \frac{k^{4} r^{3} \left(p^{4} + 10 p^{2} q^{2} + q^{4}\right)}{288} + \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{288}
\big)


In [43]:
F4 = clip(((T(p, 1)**2 + T(q, 1)**2) -\
           (T(p, 0)**2 + T(q, 0)**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^4 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

2 \left(p p_{t} + q q_{t}\right)
+h\big(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\big)
+h^2\big(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\big)
+h^3\big(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\big)
+h^5\big(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\big)


In [44]:
f4 = NF(F4, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


k**4*p*q*r*(p - q)*(p + q)*(p**2 + q**2)/24

h^5 =>


-k**4*r**2*(p - q)**2*(p + q)**2*(p**2 + q**2)/72 + k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/144 - r**6*(p**2 + q**2)/72

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{4} p q r \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{24}
\big)
+h^5\big(
- \frac{k^{4} r^{2} \left(p - q\right)^{2} \left(p + q\right)^{2} \left(p^{2} + q^{2}\right)}{72} + \frac{k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{144} - \frac{r^{6} \left(p^{2} + q^{2}\right)}{72}
\big)


In [45]:
replace(str(f4[4].collect(k)))

'k**4*(y[0]**5*y[1]*y[2]/24 - y[0]*y[1]**5*y[2]/24)'

In [46]:
F5 = clip(((k**2*T(p, 1)**2 + T(r, 1)**2) -\
           (k**2*T(p, 0)**2 + T(r, 0)**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\big(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\big)
+h^2\big(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\big)
+h^3\big(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\big)
+h^5\big(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\big)


In [47]:
f5 = NF(F5, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f5)
prnlatex(f5)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


k**2*p*q*r*(k*p - r)*(k*p + r)*(k**2*p**2 + r**2)/24

h^5 =>


-k**2*q**2*(k**2*p**2 + r**2)*(2*k**4*p**4 - k**4*p**2*q**2 + 2*k**4*q**4 - 4*k**2*p**2*r**2 + k**2*q**2*r**2 + 2*r**4)/144

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{2} p q r \left(k p - r\right) \left(k p + r\right) \left(k^{2} p^{2} + r^{2}\right)}{24}
\big)
+h^5\big(
- \frac{k^{2} q^{2} \left(k^{2} p^{2} + r^{2}\right) \left(2 k^{4} p^{4} - k^{4} p^{2} q^{2} + 2 k^{4} q^{4} - 4 k^{2} p^{2} r^{2} + k^{2} q^{2} r^{2} + 2 r^{4}\right)}{144}
\big)


In [48]:
replace(str(f5[4].factor()))

'k**2*(k*y[0] - y[2])*(k*y[0] + y[2])*(k**2*y[0]**2 + y[2]**2)*y[0]*y[1]*y[2]/24'

## Dormand-Prince 5 order

In [49]:
set_clip(8, 7, Rational(0, 1))
m = DormandPrince5(f, Matrix([p, q, r]))

In [50]:
F1 = clip((T(p, 1)-T(p, 0))/h - T(m[0], 0))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


k**2*p*q**2/2 + (p*r**2 + p_{tt})/2

h^2 =>


...

h^5 =>


k**6*p*q**2*(480*p**4 - 1327*p**2*q**2 + 36*q**4)/21600 + k**4*p*r**2*(720*p**4 - 14783*p**2*q**2 + 6124*q**4)/32400 - k**2*p*r**4*(3981*p**2 - 12248*q**2)/64800 + (6*p*r**6 + 5*p_{tttttt})/3600

h^6 =>


-k**6*p**2*q*r*(321840*p**4 - 10243019*p**2*q**2 + 4313225*q**4)/64152000 + k**4*q*r**3*(30729057*p**4 - 72952646*p**2*q**2 + 4982175*q**4)/192456000 - k**2*q*r**5*(172529*p**2 - 66429*q**2)/2566080 + p_{ttttttt}/5040

p_{t} - q r
+h\big(
\frac{k^{2} p q^{2}}{2} + \frac{p r^{2} + p_{tt}}{2}
\big)
+h^2\big(
- \frac{k^{2} q r \left(2 p - q\right) \left(2 p + q\right)}{6} + \frac{p_{ttt} + q r^{3}}{6}
\big)
+h^3\big(
\frac{k^{4} p q^{2} \cdot \left(2 p - q\right) \left(2 p + q\right)}{24} + \frac{k^{2} p r^{2} \cdot \left(2 p^{2} - 7 q^{2}\right)}{12} - \frac{p r^{4} - p_{tttt}}{24}
\big)
+h^4\big(
- \frac{k^{4} q r \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{120} + \frac{k^{2} q r^{3} \cdot \left(22 p^{2} - 7 q^{2}\right)}{60} - \frac{- p_{ttttt} + q r^{5}}{120}
\big)
+h^5\big(
\frac{k^{6} p q^{2} \cdot \left(480 p^{4} - 1327 p^{2} q^{2} + 36 q^{4}\right)}{21600} + \frac{k^{4} p r^{2} \cdot \left(720 p^{4} - 14783 p^{2} q^{2} + 6124 q^{4}\right)}{32400} - \frac{k^{2} p r^{4} \cdot \left(3981 p^{2} - 12248 q^{2}\right)}{64800} + \frac{6 p r^{6} + 5 p_{tttttt}}{3600}
\big)
+h^6\big(
- \frac{k^{6} p^{2} q r \left(321840 p^{4} - 10243019 p^{2} q^{2} + 4313225 q^{4}\right)}{64

In [51]:
F2 = clip((T(q, 1)-T(q, 0))/h - T(m[1], 0))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


-k**2*p**2*q/2 + (q*r**2 + q_{tt})/2

h^2 =>


...

h^5 =>


-k**6*p**2*q*(36*p**4 - 1327*p**2*q**2 + 480*q**4)/21600 + k**4*q*r**2*(6124*p**4 - 14783*p**2*q**2 + 720*q**4)/32400 - k**2*q*r**4*(12248*p**2 - 3981*q**2)/64800 + (6*q*r**6 + 5*q_{tttttt})/3600

h^6 =>


-k**6*p*q**2*r*(4313225*p**4 - 10243019*p**2*q**2 + 321840*q**4)/64152000 - k**4*p*r**3*(4982175*p**4 - 72952646*p**2*q**2 + 30729057*q**4)/192456000 + k**2*p*r**5*(66429*p**2 - 172529*q**2)/2566080 + q_{ttttttt}/5040

p r + q_{t}
+h\big(
- \frac{k^{2} p^{2} q}{2} + \frac{q r^{2} + q_{tt}}{2}
\big)
+h^2\big(
\frac{k^{2} p r \left(p - 2 q\right) \left(p + 2 q\right)}{6} - \frac{p r^{3} - q_{ttt}}{6}
\big)
+h^3\big(
- \frac{k^{4} p^{2} q \left(p - 2 q\right) \left(p + 2 q\right)}{24} + \frac{k^{2} q r^{2} \cdot \left(7 p^{2} - 2 q^{2}\right)}{12} - \frac{q r^{4} - q_{tttt}}{24}
\big)
+h^4\big(
\frac{k^{4} p r \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{120} - \frac{k^{2} p r^{3} \cdot \left(7 p^{2} - 22 q^{2}\right)}{60} + \frac{p r^{5} + q_{ttttt}}{120}
\big)
+h^5\big(
- \frac{k^{6} p^{2} q \left(36 p^{4} - 1327 p^{2} q^{2} + 480 q^{4}\right)}{21600} + \frac{k^{4} q r^{2} \cdot \left(6124 p^{4} - 14783 p^{2} q^{2} + 720 q^{4}\right)}{32400} - \frac{k^{2} q r^{4} \cdot \left(12248 p^{2} - 3981 q^{2}\right)}{64800} + \frac{6 q r^{6} + 5 q_{tttttt}}{3600}
\big)
+h^6\big(
- \frac{k^{6} p q^{2} r \left(4313225 p^{4} - 10243019 p^{2} q^{2} + 321840 q^{4}\right)}{64152000} - \f

In [52]:
F3 = clip((T(r, 1)-T(r, 0))/h - T(m[2], 0))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


-k**2*r*(p - q)*(p + q)/2 + r_{tt}/2

h^2 =>


...

h^5 =>


-k**6*r*(p - q)*(p + q)*(27*p**4 - 3035*p**2*q**2 + 27*q**4)/16200 + k**4*r**3*(3981*p**4 - 29566*p**2*q**2 + 3981*q**4)/64800 - k**2*r**5*(p - q)*(p + q)/45 + r_{tttttt}/720

h^6 =>


-671*k**8*p**3*q**3*(p - q)*(p + q)/25920 - k**6*p*q*r**2*(12939675*p**4 - 72952646*p**2*q**2 + 12939675*q**4)/192456000 + 10243019*k**4*p*q*r**4*(p - q)*(p + q)/64152000 - 149*k**2*p*q*r**6/29700 + r_{ttttttt}/5040

k^{2} p q + r_{t}
+h\big(
- \frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2} + \frac{r_{tt}}{2}
\big)
+h^2\big(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{6} - \frac{2 k^{2} p q r^{2}}{3} + \frac{r_{ttt}}{6}
\big)
+h^3\big(
- \frac{k^{4} r \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{24} + \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{6} + \frac{r_{tttt}}{24}
\big)
+h^4\big(
\frac{k^{6} p q \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{120} - \frac{11 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{30} + \frac{2 k^{2} p q r^{4}}{15} + \frac{r_{ttttt}}{120}
\big)
+h^5\big(
- \frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(27 p^{4} - 3035 p^{2} q^{2} + 27 q^{4}\right)}{16200} + \frac{k^{4} r^{3} \cdot \left(3981 p^{4} - 29566 p^{2} q^{2} + 3981 q^{4}\right)}{64800} - \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{45} + \frac{r_{tttttt}}{720}
\big)
+h^6\big(
- \frac{671 k^{8} p^{3} 

In [53]:
f1 = NF(F1, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^5 =>


-k**6*p*q**4*(7*p**2 - 6*q**2)/21600 - k**4*p*q**2*r**2*(23*p**2 - 49*q**2)/32400 - 7*k**2*p*r**4*(3*p**2 - 14*q**2)/64800 + p*r**6/3600

h^6 =>


k**6*q*r*(1787130*p**6 - 5290121*p**4*q**2 + 3885725*p**2*q**4 - 106920*q**6)/224532000 - k**4*q*r**3*(15870363*p**4 - 25789604*p**2*q**2 + 1223640*q**4)/673596000 + k**2*q*r**5*(777145*p**2 - 81576*q**2)/44906400 - q*r**7/2100

p_{t} - q r
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
- \frac{k^{6} p q^{4} \cdot \left(7 p^{2} - 6 q^{2}\right)}{21600} - \frac{k^{4} p q^{2} r^{2} \cdot \left(23 p^{2} - 49 q^{2}\right)}{32400} - \frac{7 k^{2} p r^{4} \cdot \left(3 p^{2} - 14 q^{2}\right)}{64800} + \frac{p r^{6}}{3600}
\big)
+h^6\big(
\frac{k^{6} q r \left(1787130 p^{6} - 5290121 p^{4} q^{2} + 3885725 p^{2} q^{4} - 106920 q^{6}\right)}{224532000} - \frac{k^{4} q r^{3} \cdot \left(15870363 p^{4} - 25789604 p^{2} q^{2} + 1223640 q^{4}\right)}{673596000} + \frac{k^{2} q r^{5} \cdot \left(777145 p^{2} - 81576 q^{2}\right)}{44906400} - \frac{q r^{7}}{2100}
\big)


In [54]:
f2 = NF(F2, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-k**6*p**4*q*(6*p**2 - 7*q**2)/21600 + k**4*p**2*q*r**2*(49*p**2 - 23*q**2)/32400 - 7*k**2*q*r**4*(14*p**2 - 3*q**2)/64800 + q*r**6/3600

h^6 =>


-k**6*p*r*(106920*p**6 - 3885725*p**4*q**2 + 5290121*p**2*q**4 - 1787130*q**6)/224532000 + k**4*p*r**3*(1223640*p**4 - 25789604*p**2*q**2 + 15870363*q**4)/673596000 - k**2*p*r**5*(81576*p**2 - 777145*q**2)/44906400 + p*r**7/2100

p r + q_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
- \frac{k^{6} p^{4} q \left(6 p^{2} - 7 q^{2}\right)}{21600} + \frac{k^{4} p^{2} q r^{2} \cdot \left(49 p^{2} - 23 q^{2}\right)}{32400} - \frac{7 k^{2} q r^{4} \cdot \left(14 p^{2} - 3 q^{2}\right)}{64800} + \frac{q r^{6}}{3600}
\big)
+h^6\big(
- \frac{k^{6} p r \left(106920 p^{6} - 3885725 p^{4} q^{2} + 5290121 p^{2} q^{4} - 1787130 q^{6}\right)}{224532000} + \frac{k^{4} p r^{3} \cdot \left(1223640 p^{4} - 25789604 p^{2} q^{2} + 15870363 q^{4}\right)}{673596000} - \frac{k^{2} p r^{5} \cdot \left(81576 p^{2} - 777145 q^{2}\right)}{44906400} + \frac{p r^{7}}{2100}
\big)


In [55]:
f3 = NF(F3, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-k**6*r*(p - q)*(p + q)*(9*p**4 - 40*p**2*q**2 + 9*q**4)/32400 + k**4*r**3*(21*p**4 - 46*p**2*q**2 + 21*q**4)/64800

h^6 =>


-k**8*p*q*(p - q)*(p + q)*(27*p**4 - 76*p**2*q**2 + 27*q**4)/56700 + k**6*p*q*r**2*(11657175*p**4 - 25789604*p**2*q**2 + 11657175*q**4)/673596000 - 5290121*k**4*p*q*r**4*(p - q)*(p + q)/224532000 + 6619*k**2*p*q*r**6/831600

k^{2} p q + r_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
- \frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(9 p^{4} - 40 p^{2} q^{2} + 9 q^{4}\right)}{32400} + \frac{k^{4} r^{3} \cdot \left(21 p^{4} - 46 p^{2} q^{2} + 21 q^{4}\right)}{64800}
\big)
+h^6\big(
- \frac{k^{8} p q \left(p - q\right) \left(p + q\right) \left(27 p^{4} - 76 p^{2} q^{2} + 27 q^{4}\right)}{56700} + \frac{k^{6} p q r^{2} \cdot \left(11657175 p^{4} - 25789604 p^{2} q^{2} + 11657175 q^{4}\right)}{673596000} - \frac{5290121 k^{4} p q r^{4} \left(p - q\right) \left(p + q\right)}{224532000} + \frac{6619 k^{2} p q r^{6}}{831600}
\big)


In [56]:
F4 = clip(((T(p, 1)**2 + T(q, 1)**2) -\
           (T(p, 0)**2 + T(q, 0)**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

h^6 =>


(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt} + q*q_{ttttttt} + 7*q_{tttttt}*q_{t} + 21*q_{ttttt}*q_{tt} + 35*q_{tttt}*q_{ttt})/2520

2 \left(p p_{t} + q q_{t}\right)
+h\big(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\big)
+h^2\big(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\big)
+h^3\big(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\big)
+h^5\big(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\big)
+h^6\big(
\frac{p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt} + q q_{ttttttt} + 7 q_{tttttt} q_{t} + 21 q_{ttttt} q_{tt} + 35 q_{tttt} q_{ttt}}{2520}
\big)


In [57]:
f4 = NF(F4, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


k**6*p**2*q**2*(p - q)*(p + q)*(p**2 + q**2)/1800 - 13*k**4*p**2*q**2*r**2*(p**2 + q**2)/8100 + 7*k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/10800 - r**6*(p**2 + q**2)/1800

h^6 =>


-k**6*p*q*r*(p**2 + q**2)*(4610*p**4 - 9297*p**2*q**2 + 4610*q**4)/297000 + 702413*k**4*p*q*r**3*(p - q)*(p + q)*(p**2 + q**2)/16038000 - 89863*k**2*p*q*r**5*(p**2 + q**2)/3207600

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
\frac{k^{6} p^{2} q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{1800} - \frac{13 k^{4} p^{2} q^{2} r^{2} \left(p^{2} + q^{2}\right)}{8100} + \frac{7 k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{10800} - \frac{r^{6} \left(p^{2} + q^{2}\right)}{1800}
\big)
+h^6\big(
- \frac{k^{6} p q r \left(p^{2} + q^{2}\right) \left(4610 p^{4} - 9297 p^{2} q^{2} + 4610 q^{4}\right)}{297000} + \frac{702413 k^{4} p q r^{3} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{16038000} - \frac{89863 k^{2} p q r^{5} \left(p^{2} + q^{2}\right)}{3207600}
\big)


In [58]:
replace(str(f4[5].collect(k)))

'k**6*(y[0]**6*y[1]**2/1800 - y[0]**2*y[1]**6/1800) + k**4*(-13*y[0]**4*y[1]**2*y[2]**2/8100 - 13*y[0]**2*y[1]**4*y[2]**2/8100) + k**2*(7*y[0]**4*y[2]**4/10800 - 7*y[1]**4*y[2]**4/10800) - y[0]**2*y[2]**6/1800 - y[1]**2*y[2]**6/1800'

In [59]:
F5 = clip(((k**2*T(p, 1)**2 + T(r, 1)**2) -\
           (k**2*T(p, 0)**2 + T(r, 0)**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

h^6 =>


k**2*(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt})/2520 + (r*r_{ttttttt} + 7*r_{tttttt}*r_{t} + 21*r_{ttttt}*r_{tt} + 35*r_{tttt}*r_{ttt})/2520

2 k^{2} p p_{t} + 2 r r_{t}
+h\big(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\big)
+h^2\big(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\big)
+h^3\big(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\big)
+h^5\big(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\big)
+h^6\big(
\frac{k^{2} \left(p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt}\right)}{2520} + \frac{r r_{ttttttt} + 7 r_{tttttt} r_{t} + 21 r_{ttttt} r_{tt} + 35 r_{tttt} r_{ttt}}{2520}
\big)


In [60]:
f5 = NF(F5, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


k**8*p**2*q**4*(7*p**2 - 6*q**2)/10800 + k**6*r**2*(p**2 - 3*q**2)*(9*p**4 + p**2*q**2 + 3*q**4)/16200 - k**4*q**2*r**4*(52*p**2 + 21*q**2)/32400 - k**2*p**2*r**6/1800

h^6 =>


-k**8*p**3*q*r*(248940*p**4 - 702413*p**2*q**2 + 449315*q**4)/16038000 + k**6*p*q*r**3*(253098*p**4 - 449315*q**4)/16038000 + k**4*p*q*r**5*(253098*p**2 - 702413*q**2)/16038000 - 461*k**2*p*q*r**7/29700

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
\frac{k^{8} p^{2} q^{4} \cdot \left(7 p^{2} - 6 q^{2}\right)}{10800} + \frac{k^{6} r^{2} \left(p^{2} - 3 q^{2}\right) \left(9 p^{4} + p^{2} q^{2} + 3 q^{4}\right)}{16200} - \frac{k^{4} q^{2} r^{4} \cdot \left(52 p^{2} + 21 q^{2}\right)}{32400} - \frac{k^{2} p^{2} r^{6}}{1800}
\big)
+h^6\big(
- \frac{k^{8} p^{3} q r \left(248940 p^{4} - 702413 p^{2} q^{2} + 449315 q^{4}\right)}{16038000} + \frac{k^{6} p q r^{3} \cdot \left(253098 p^{4} - 449315 q^{4}\right)}{16038000} + \frac{k^{4} p q r^{5} \cdot \left(253098 p^{2} - 702413 q^{2}\right)}{16038000} - \frac{461 k^{2} p q r^{7}}{29700}
\big)


In [61]:
replace(str(f5[5].collect(k)))

'k**8*(7*y[0]**4*y[1]**4/10800 - y[0]**2*y[1]**6/1800) + k**6*(y[0]**6*y[2]**2/1800 - 13*y[0]**4*y[1]**2*y[2]**2/8100 - y[1]**6*y[2]**2/1800) + k**4*(-13*y[0]**2*y[1]**2*y[2]**4/8100 - 7*y[1]**4*y[2]**4/10800) - k**2*y[0]**2*y[2]**6/1800'

## Dormand-Prince 4 order

In [62]:
set_clip(7, 6, Rational(0, 1))
m = DormandPrince51(f, Matrix([p, q, r]))

In [63]:
F1 = clip((T(p, 1)-T(p, 0))/h - T(m[0], 0))
prn(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


k**2*p*q**2/2 + (p*r**2 + p_{tt})/2

h^2 =>


...

k**4*p*q**2*(2*p - q)*(2*p + q)/24 + k**2*p*r**2*(2*p**2 - 7*q**2)/12 - (p*r**4 - p_{tttt})/24

h^4 =>


-k**4*q*r*(431913*p**4 - 1197259*p**2*q**2 + 29619*q**4)/3240000 + k**2*q*r**3*(1197259*p**2 - 385023*q**2)/3240000 - (-1000*p_{ttttt} + 1097*q*r**5)/120000

h^5 =>


k**6*p*q**2*(718336*p**4 - 2000899*p**2*q**2 + 43470*q**4)/32400000 + k**4*p*r**2*(3232512*p**4 - 68009893*p**2*q**2 + 28119005*q**4)/145800000 - k**2*p*r**4*(18008091*p**2 - 56238010*q**2)/291600000 + (483*p*r**6 + 500*p_{tttttt})/360000

In [64]:
F2 = clip((T(q, 1)-T(q, 0))/h - T(m[1], 0))
prn(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


-k**2*p**2*q/2 + (q*r**2 + q_{tt})/2

h^2 =>


...

-k**4*p**2*q*(p - 2*q)*(p + 2*q)/24 + k**2*q*r**2*(7*p**2 - 2*q**2)/12 - (q*r**4 - q_{tttt})/24

h^4 =>


k**4*p*r*(29619*p**4 - 1197259*p**2*q**2 + 431913*q**4)/3240000 - k**2*p*r**3*(385023*p**2 - 1197259*q**2)/3240000 + (1097*p*r**5 + 1000*q_{ttttt})/120000

h^5 =>


-k**6*p**2*q*(43470*p**4 - 2000899*p**2*q**2 + 718336*q**4)/32400000 + k**4*q*r**2*(28119005*p**4 - 68009893*p**2*q**2 + 3232512*q**4)/145800000 - k**2*q*r**4*(56238010*p**2 - 18008091*q**2)/291600000 + (483*q*r**6 + 500*q_{tttttt})/360000

In [65]:
F3 = clip((T(r, 1)-T(r, 0))/h - T(m[2], 0))
prn(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


-k**2*r*(p - q)*(p + q)/2 + r_{tt}/2

h^2 =>


...

-k**4*r*(p**2 - 4*p*q + q**2)*(p**2 + 4*p*q + q**2)/24 + k**2*r**3*(p - q)*(p + q)/6 + r_{tttt}/24

h^4 =>


k**6*p*q*(9873*p**4 - 128341*p**2*q**2 + 9873*q**4)/1080000 - 1197259*k**4*p*q*r**2*(p - q)*(p + q)/3240000 + 143971*k**2*p*q*r**4/1080000 + r_{ttttt}/120

h^5 =>


-k**6*r*(p - q)*(p + q)*(39123*p**4 - 5584678*p**2*q**2 + 39123*q**4)/29160000 + k**4*r**3*(18008091*p**4 - 136019786*p**2*q**2 + 18008091*q**4)/291600000 - 5612*k**2*r**5*(p - q)*(p + q)/253125 + r_{tttttt}/720

In [66]:
f1 = NF(F1, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


k**4*q*r*(87*p**4 + 9259*p**2*q**2 - 2619*q**4)/3240000 + k**2*q*r**3*(9259*p**2 - 7023*q**2)/3240000 - 97*q*r**5/120000

h^5 =>


-k**6*p*q**2*(14324*p**4 - 60511*p**2*q**2 + 27720*q**4)/32400000 - k**4*p*r**2*(64458*p**4 - 68897*p**2*q**2 + 610210*q**4)/145800000 + k**2*p*r**4*(544599*p**2 - 1220420*q**2)/291600000 - 77*p*r**6/90000

p_{t} - q r
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{4} q r \left(87 p^{4} + 9259 p^{2} q^{2} - 2619 q^{4}\right)}{3240000} + \frac{k^{2} q r^{3} \cdot \left(9259 p^{2} - 7023 q^{2}\right)}{3240000} - \frac{97 q r^{5}}{120000}
\big)
+h^5\big(
- \frac{k^{6} p q^{2} \cdot \left(14324 p^{4} - 60511 p^{2} q^{2} + 27720 q^{4}\right)}{32400000} - \frac{k^{4} p r^{2} \cdot \left(64458 p^{4} - 68897 p^{2} q^{2} + 610210 q^{4}\right)}{145800000} + \frac{k^{2} p r^{4} \cdot \left(544599 p^{2} - 1220420 q^{2}\right)}{291600000} - \frac{77 p r^{6}}{90000}
\big)


In [67]:
f2 = NF(F2, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


k**4*p*r*(2619*p**4 - 9259*p**2*q**2 - 87*q**4)/3240000 - k**2*p*r**3*(7023*p**2 - 9259*q**2)/3240000 + 97*p*r**5/120000

h^5 =>


k**6*p**2*q*(27720*p**4 - 60511*p**2*q**2 + 14324*q**4)/32400000 - k**4*q*r**2*(610210*p**4 - 68897*p**2*q**2 + 64458*q**4)/145800000 + k**2*q*r**4*(1220420*p**2 - 544599*q**2)/291600000 - 77*q*r**6/90000

p r + q_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{4} p r \left(2619 p^{4} - 9259 p^{2} q^{2} - 87 q^{4}\right)}{3240000} - \frac{k^{2} p r^{3} \cdot \left(7023 p^{2} - 9259 q^{2}\right)}{3240000} + \frac{97 p r^{5}}{120000}
\big)
+h^5\big(
\frac{k^{6} p^{2} q \left(27720 p^{4} - 60511 p^{2} q^{2} + 14324 q^{4}\right)}{32400000} - \frac{k^{4} q r^{2} \cdot \left(610210 p^{4} - 68897 p^{2} q^{2} + 64458 q^{4}\right)}{145800000} + \frac{k^{2} q r^{4} \cdot \left(1220420 p^{2} - 544599 q^{2}\right)}{291600000} - \frac{77 q r^{6}}{90000}
\big)


In [68]:
f3 = NF(F3, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


k**6*p*q*(873*p**4 - 2341*p**2*q**2 + 873*q**4)/1080000 - 9259*k**4*p*q*r**2*(p - q)*(p + q)/3240000 - 29*k**2*p*q*r**4/1080000

h^5 =>


k**6*r*(p - q)*(p + q)*(12474*p**4 - 48547*p**2*q**2 + 12474*q**4)/14580000 - k**4*r**3*(544599*p**4 - 137794*p**2*q**2 + 544599*q**4)/291600000 + 3581*k**2*r**5*(p - q)*(p + q)/8100000

k^{2} p q + r_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{6} p q \left(873 p^{4} - 2341 p^{2} q^{2} + 873 q^{4}\right)}{1080000} - \frac{9259 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{3240000} - \frac{29 k^{2} p q r^{4}}{1080000}
\big)
+h^5\big(
\frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(12474 p^{4} - 48547 p^{2} q^{2} + 12474 q^{4}\right)}{14580000} - \frac{k^{4} r^{3} \cdot \left(544599 p^{4} - 137794 p^{2} q^{2} + 544599 q^{4}\right)}{291600000} + \frac{3581 k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{8100000}
\big)


In [69]:
F4 = clip((T(p, 1)**2 + T(q, 1)**2) -\
          (T(p, 0)**2 + T(q, 0)**2))
prn(F4, k)
prnlatex(F4, k)

h^0 =>


0

h^1 =>


2*(p*p_{t} + q*q_{t})

h^2 =>


...

h^4 =>


(p*p_{tttt} + 4*p_{ttt}*p_{t} + 3*p_{tt}**2 + q*q_{tttt} + 4*q_{ttt}*q_{t} + 3*q_{tt}**2)/12

h^5 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

0
+h\big(
2 \left(p p_{t} + q q_{t}\right)
\big)
+h^2\big(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\big)
+h^3\big(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\big)
+h^4\big(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\big)
+h^5\big(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\big)


In [70]:
f4 = NF(F4, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-451*k**4*p*q*r*(p - q)*(p + q)*(p**2 + q**2)/270000 - 559*k**2*p*q*r**3*(p**2 + q**2)/405000

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
- \frac{451 k^{4} p q r \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{270000} - \frac{559 k^{2} p q r^{3} \left(p^{2} + q^{2}\right)}{405000}
\big)


In [71]:
replace(str(f4[4].collect(k)))

'0'

In [72]:
F5 = clip(((k**2*T(p, 1)**2 + T(r, 1)**2) -\
           (k**2*T(p, 0)**2 + T(r, 0)**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\big(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\big)
+h^2\big(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\big)
+h^3\big(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\big)
+h^5\big(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\big)


In [73]:
f5 = NF(F5, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


-k**6*p**3*q*r*(1353*p**2 + 1118*q**2)/810000 - 559*k**4*p*q**3*r**3/405000 + 451*k**2*p*q*r**5/270000

h^5 =>


k**8*p**2*q**2*(27854*p**4 - 49331*p**2*q**2 + 27720*q**4)/16200000 + k**6*r**2*(603*p**6 + 387818*p**4*q**2 + 124740*q**6)/72900000 + k**4*q**2*r**4*(775636*p**2 + 443979*q**2)/145800000 - k**2*r**6*(67*p**2 - 13927*q**2)/8100000

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{k^{6} p^{3} q r \left(1353 p^{2} + 1118 q^{2}\right)}{810000} - \frac{559 k^{4} p q^{3} r^{3}}{405000} + \frac{451 k^{2} p q r^{5}}{270000}
\big)
+h^5\big(
\frac{k^{8} p^{2} q^{2} \cdot \left(27854 p^{4} - 49331 p^{2} q^{2} + 27720 q^{4}\right)}{16200000} + \frac{k^{6} r^{2} \cdot \left(603 p^{6} + 387818 p^{4} q^{2} + 124740 q^{6}\right)}{72900000} + \frac{k^{4} q^{2} r^{4} \cdot \left(775636 p^{2} + 443979 q^{2}\right)}{145800000} - \frac{k^{2} r^{6} \cdot \left(67 p^{2} - 13927 q^{2}\right)}{8100000}
\big)


In [74]:
replace(str(f5[4].collect(k)))

'k**6*(-451*y[0]**5*y[1]*y[2]/270000 - 559*y[0]**3*y[1]**3*y[2]/405000) - 559*k**4*y[0]*y[1]**3*y[2]**3/405000 + 451*k**2*y[0]*y[1]*y[2]**5/270000'

## The implicit midpoint rule for elliptic functions

In [75]:
set_clip(8, 7, Rational(0, 1))

In [76]:
F1 = clip(4*(T(p, 1) - T(p, 0))/h - (T(q, 1) + T(q, 0))*(T(r, 1) + T(r, 0)))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


-4*(-p_{t} + q*r)

h^1 =>


-2*(-p_{tt} + q*r_{t} + q_{t}*r)

h^2 =>


...

h^5 =>


-(-2*p_{tttttt} + 6*q*r_{ttttt} + 6*q_{ttttt}*r + 15*q_{tttt}*r_{t} + 30*q_{ttt}*r_{tt} + 30*q_{tt}*r_{ttt} + 15*q_{t}*r_{tttt})/360

h^6 =>


-(-4*p_{ttttttt} + 14*q*r_{tttttt} + 14*q_{tttttt}*r + 42*q_{ttttt}*r_{t} + 105*q_{tttt}*r_{tt} + 140*q_{ttt}*r_{ttt} + 105*q_{tt}*r_{tttt} + 42*q_{t}*r_{ttttt})/5040

- 4 \left(- p_{t} + q r\right)
+h\big(
- 2 \left(- p_{tt} + q r_{t} + q_{t} r\right)
\big)
+h^2\big(
- \frac{- 2 p_{ttt} + 3 q r_{tt} + 3 q_{tt} r + 3 q_{t} r_{t}}{3}
\big)
+h^3\big(
- \frac{- p_{tttt} + 2 q r_{ttt} + 2 q_{ttt} r + 3 q_{tt} r_{t} + 3 q_{t} r_{tt}}{6}
\big)
+h^4\big(
- \frac{- 2 p_{ttttt} + 5 q r_{tttt} + 5 q_{tttt} r + 10 q_{ttt} r_{t} + 15 q_{tt} r_{tt} + 10 q_{t} r_{ttt}}{60}
\big)
+h^5\big(
- \frac{- 2 p_{tttttt} + 6 q r_{ttttt} + 6 q_{ttttt} r + 15 q_{tttt} r_{t} + 30 q_{ttt} r_{tt} + 30 q_{tt} r_{ttt} + 15 q_{t} r_{tttt}}{360}
\big)
+h^6\big(
- \frac{- 4 p_{ttttttt} + 14 q r_{tttttt} + 14 q_{tttttt} r + 42 q_{ttttt} r_{t} + 105 q_{tttt} r_{tt} + 140 q_{ttt} r_{ttt} + 105 q_{tt} r_{tttt} + 42 q_{t} r_{ttttt}}{5040}
\big)


In [77]:
F2 = clip(4*(T(q, 1) - T(q, 0))/h + (T(p, 1) + T(p, 0))*(T(r, 1) + T(r, 0)))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


4*(p*r + q_{t})

h^1 =>


2*(p*r_{t} + p_{t}*r + q_{tt})

h^2 =>


...

h^5 =>


(6*p*r_{ttttt} + 6*p_{ttttt}*r + 15*p_{tttt}*r_{t} + 30*p_{ttt}*r_{tt} + 30*p_{tt}*r_{ttt} + 15*p_{t}*r_{tttt} + 2*q_{tttttt})/360

h^6 =>


(14*p*r_{tttttt} + 14*p_{tttttt}*r + 42*p_{ttttt}*r_{t} + 105*p_{tttt}*r_{tt} + 140*p_{ttt}*r_{ttt} + 105*p_{tt}*r_{tttt} + 42*p_{t}*r_{ttttt} + 4*q_{ttttttt})/5040

4 \left(p r + q_{t}\right)
+h\big(
2 \left(p r_{t} + p_{t} r + q_{tt}\right)
\big)
+h^2\big(
\frac{3 p r_{tt} + 3 p_{tt} r + 3 p_{t} r_{t} + 2 q_{ttt}}{3}
\big)
+h^3\big(
\frac{2 p r_{ttt} + 2 p_{ttt} r + 3 p_{tt} r_{t} + 3 p_{t} r_{tt} + q_{tttt}}{6}
\big)
+h^4\big(
\frac{5 p r_{tttt} + 5 p_{tttt} r + 10 p_{ttt} r_{t} + 15 p_{tt} r_{tt} + 10 p_{t} r_{ttt} + 2 q_{ttttt}}{60}
\big)
+h^5\big(
\frac{6 p r_{ttttt} + 6 p_{ttttt} r + 15 p_{tttt} r_{t} + 30 p_{ttt} r_{tt} + 30 p_{tt} r_{ttt} + 15 p_{t} r_{tttt} + 2 q_{tttttt}}{360}
\big)
+h^6\big(
\frac{14 p r_{tttttt} + 14 p_{tttttt} r + 42 p_{ttttt} r_{t} + 105 p_{tttt} r_{tt} + 140 p_{ttt} r_{ttt} + 105 p_{tt} r_{tttt} + 42 p_{t} r_{ttttt} + 4 q_{ttttttt}}{5040}
\big)


In [78]:
F3 = clip(4*(T(r, 1) - T(r, 0))/h + k**2*(T(p, 1) + T(p, 0))*(T(q, 1) + T(q, 0)))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


4*k**2*p*q + 4*r_{t}

h^1 =>


2*k**2*(p*q_{t} + p_{t}*q) + 2*r_{tt}

h^2 =>


...

h^5 =>


k**2*(2*p*q_{ttttt} + 2*p_{ttttt}*q + 5*p_{tttt}*q_{t} + 10*p_{ttt}*q_{tt} + 10*p_{tt}*q_{ttt} + 5*p_{t}*q_{tttt})/120 + r_{tttttt}/180

h^6 =>


k**2*(2*p*q_{tttttt} + 2*p_{tttttt}*q + 6*p_{ttttt}*q_{t} + 15*p_{tttt}*q_{tt} + 20*p_{ttt}*q_{ttt} + 15*p_{tt}*q_{tttt} + 6*p_{t}*q_{ttttt})/720 + r_{ttttttt}/1260

4 k^{2} p q + 4 r_{t}
+h\big(
2 k^{2} \left(p q_{t} + p_{t} q\right) + 2 r_{tt}
\big)
+h^2\big(
k^{2} \left(p q_{tt} + p_{tt} q + p_{t} q_{t}\right) + \frac{2 r_{ttt}}{3}
\big)
+h^3\big(
\frac{k^{2} \cdot \left(2 p q_{ttt} + 2 p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{6} + \frac{r_{tttt}}{6}
\big)
+h^4\big(
\frac{k^{2} \left(p q_{tttt} + p_{tttt} q + 2 p_{ttt} q_{t} + 3 p_{tt} q_{tt} + 2 p_{t} q_{ttt}\right)}{12} + \frac{r_{ttttt}}{30}
\big)
+h^5\big(
\frac{k^{2} \cdot \left(2 p q_{ttttt} + 2 p_{ttttt} q + 5 p_{tttt} q_{t} + 10 p_{ttt} q_{tt} + 10 p_{tt} q_{ttt} + 5 p_{t} q_{tttt}\right)}{120} + \frac{r_{tttttt}}{180}
\big)
+h^6\big(
\frac{k^{2} \cdot \left(2 p q_{tttttt} + 2 p_{tttttt} q + 6 p_{ttttt} q_{t} + 15 p_{tttt} q_{tt} + 20 p_{ttt} q_{ttt} + 15 p_{tt} q_{tttt} + 6 p_{t} q_{ttttt}\right)}{720} + \frac{r_{ttttttt}}{1260}
\big)


In [79]:
f1 = NF(F1, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


-4*(-p_{t} + q*r)

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


-k**6*q*r*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/1680 + k**4*q*r**3*(29*p**4 + 4*p**2*q**2 + 29*q**4)/840 - 29*k**2*q*r**5*(p - q)*(p + q)/840 + q*r**7/112

- 4 \left(- p_{t} + q r\right)
+h\big(
0
\big)
+h^2\big(
- \frac{k^{2} q r \left(p - q\right) \left(p + q\right)}{3} + \frac{q r^{3}}{3}
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{k^{4} q r \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{60} + \frac{7 k^{2} q r^{3} \left(p - q\right) \left(p + q\right)}{60} - \frac{q r^{5}}{20}
\big)
+h^5\big(
0
\big)
+h^6\big(
- \frac{k^{6} q r \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{1680} + \frac{k^{4} q r^{3} \cdot \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{840} - \frac{29 k^{2} q r^{5} \left(p - q\right) \left(p + q\right)}{840} + \frac{q r^{7}}{112}
\big)


In [80]:
f2 = NF(F2, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


4*(p*r + q_{t})

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


k**6*p*r*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/1680 - k**4*p*r**3*(29*p**4 + 4*p**2*q**2 + 29*q**4)/840 + 29*k**2*p*r**5*(p - q)*(p + q)/840 - p*r**7/112

4 \left(p r + q_{t}\right)
+h\big(
0
\big)
+h^2\big(
\frac{k^{2} p r \left(p - q\right) \left(p + q\right)}{3} - \frac{p r^{3}}{3}
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{4} p r \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{60} - \frac{7 k^{2} p r^{3} \left(p - q\right) \left(p + q\right)}{60} + \frac{p r^{5}}{20}
\big)
+h^5\big(
0
\big)
+h^6\big(
\frac{k^{6} p r \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{1680} - \frac{k^{4} p r^{3} \cdot \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{840} + \frac{29 k^{2} p r^{5} \left(p - q\right) \left(p + q\right)}{840} - \frac{p r^{7}}{112}
\big)


In [81]:
f3 = NF(F3, [p.diff(t), q.diff(t), r.diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


4*k**2*p*q + 4*r_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


k**8*p*q*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/1680 - k**6*p*q*r**2*(29*p**4 + 4*p**2*q**2 + 29*q**4)/840 + 29*k**4*p*q*r**4*(p - q)*(p + q)/840 - k**2*p*q*r**6/112

4 k^{2} p q + 4 r_{t}
+h\big(
0
\big)
+h^2\big(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{3} - \frac{k^{2} p q r^{2}}{3}
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{k^{6} p q \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{60} - \frac{7 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{60} + \frac{k^{2} p q r^{4}}{20}
\big)
+h^5\big(
0
\big)
+h^6\big(
\frac{k^{8} p q \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{1680} - \frac{k^{6} p q r^{2} \cdot \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{840} + \frac{29 k^{4} p q r^{4} \left(p - q\right) \left(p + q\right)}{840} - \frac{k^{2} p q r^{6}}{112}
\big)


In [82]:
F4 = clip(((T(p, 1)**2 + T(q, 1)**2) -\
           (T(p, 0)**2 + T(q, 0)**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

h^6 =>


(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt} + q*q_{ttttttt} + 7*q_{tttttt}*q_{t} + 21*q_{ttttt}*q_{tt} + 35*q_{tttt}*q_{ttt})/2520

2 \left(p p_{t} + q q_{t}\right)
+h\big(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\big)
+h^2\big(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\big)
+h^3\big(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\big)
+h^5\big(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\big)
+h^6\big(
\frac{p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt} + q q_{ttttttt} + 7 q_{tttttt} q_{t} + 21 q_{ttttt} q_{tt} + 35 q_{tttt} q_{ttt}}{2520}
\big)


In [83]:
f4 = NF(F4, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
0
\big)
+h^6\big(
0
\big)


In [84]:
F5 = clip(((k**2*T(p, 1)**2 + T(r, 1)**2) -\
           (k**2*T(p, 0)**2 + T(r, 0)**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

h^6 =>


k**2*(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt})/2520 + (r*r_{ttttttt} + 7*r_{tttttt}*r_{t} + 21*r_{ttttt}*r_{tt} + 35*r_{tttt}*r_{ttt})/2520

2 k^{2} p p_{t} + 2 r r_{t}
+h\big(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\big)
+h^2\big(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\big)
+h^3\big(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\big)
+h^4\big(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\big)
+h^5\big(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\big)
+h^6\big(
\frac{k^{2} \left(p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt}\right)}{2520} + \frac{r r_{ttttttt} + 7 r_{tttttt} r_{t} + 21 r_{ttttt} r_{tt} + 35 r_{tttt} r_{ttt}}{2520}
\big)


In [85]:
f5 = NF(F5, [p.diff(t), q.diff(t), r.diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
0
\big)
+h^6\big(
0
\big)
